# 1 - Intro bifacial_radiance modeling 

This journal shows how to model using our tracker-dictionary, which takes care of each timestamp that will be simulated. This method allows for multiple scene Objects, and multiple Analysis results. For examples of using this tracker-dictionary with Cumulative sky, please see the Jupyter tutorils.

Note that this is called a 'tracker-dictionary', but fixed systems can also be simulated.


## 0. Setup 

In [ ]:
!pip install pyradiance

In [ ]:
!pip install git+https://github.com/NREL/bifacial_radiance.git@development

In [ ]:
import bifacial_radiance as br
import numpy as np
import pandas as pd

In [ ]:
# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)

## 1. bifacial_radiance Simulation

In [ ]:
import os
from pathlib import Path

testfolder = 'Demo1'
if not os.path.exists(testfolder):
    os.makedirs(testfolder)
print ("Your simulation will be stored in %s" % testfolder)

In [ ]:
demo = br.RadianceObj('Demo1',path = testfolder)  

### Download and read the weather data

In [ ]:
epwfile = demo.getEPW(40.7608, -111.8910) 

metdata = demo.readWeatherFile(epwfile, coerce_year=2024, starttime='2024-08-27_1300', endtime='2024-08-27_1430')


In [ ]:
demo.metdata.datetime

### Set the albedo

You can pass a value for  fixed value, an existing material in ground.rad, or empty it will grab the albedo column from the weatherdata 

In [ ]:
demo.setGround(0.2) 

### Make the module

![AgriPV modeled step 4](images/spacing_between_modules.PNG)

In [ ]:
# MakeModule Parameters
numpanels = 1  # AgriPV site has 3 modules along the y direction
module_x = 1 # m
module_y = 2 # m. slope we will measure. if x>y module is in landscape-orientation.
xgap = 0.02 # m
zgap = 0.1 # m

#Add torquetube 
tubeParams = {'tubetype':'square',  # other options: 'square', 'hex', 'round'
              'diameter':0.1,  #m
              'material':'Metal_Grey', # other common option: 'black'
              'axisofrotation':True,
              'visible':True}

PVmodule=demo.makeModule(name='PVModule1',x=module_x,y=module_y,numpanels=numpanels, 
                       xgap=xgap, tubeParams=tubeParams)

#Alternative:
#PVmodule.addTorquetube(diameter=0.1, material='Metal_Grey', tubetype='square') # Object-oriented addition of features to the PV module

## Make the trackerdictionary

In [ ]:
limit_angle = 50 # tracker rotation limit angle. Setting it ridiculously small so this runs faster.
angledelta = 0.01 # sampling between the limit angles. 
backtrack = True
gcr = 0.33   # Ground coverage ratio, GCR = collector-width/pitch
cumulativesky = False 
trackerdict = demo.set1axis(metdata = metdata, limit_angle = limit_angle, backtrack = backtrack, 
                            gcr = gcr, cumulativesky = cumulativesky)

## Make the Sky

In [ ]:
demo.gendaylit1axis()

### Make the Scene

In [ ]:
sceneDict = {'pitch': 6,'hub_height':2.5, 'nMods': 3, 'nRows': 1}  
scene = demo.makeScene1axis(module=PVmodule, sceneDict=sceneDict) 

### Put it all together

In [ ]:
octfile = demo.makeOct1axis()

If desired, you can view the Oct file at this point:

***rvu -vf views\front.vp -e .01 tutorial_1.oct***

In [ ]:
## Comment the ! line below to run rvu from the Jupyter notebook instead of your terminal.
## Simulation will stop until you close the rvu window

#!rvu -vf views\front.vp -e .01 1axis_2024-08-27_1400.oct

### Analyze the Panel

In [ ]:
analysis = br.AnalysisObj()  

In [ ]:
trackerdict = demo.analysis1axis(trackerdict, modWanted=2, rowWanted = 2, sensorsy=1)

In [ ]:
results = demo.calculateResults()

In [ ]:
demo.CompiledResults